## PART 1 : Import necessary functions, set parameters and load your own recordings ##

#### Import librairies, custom functions and set repository folder ####

In [2]:
# Importation of librairies and functions
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import json
from os.path import join

In [3]:
project_path = os.getcwd()
os.chdir("C:\\Users\\Juliette\\Research\\Projects\\PyPerceive\\code")
from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

#reset the proper working directory for the analysis
os.chdir(project_path)


In [3]:
# import custom-made functions:
from utils import *
from resync_function import run_resync
from interactive import *
from tmsi_poly5reader import *
from loading_data import _load_mat_file, _load_data_lfp, _load_TMSi_artefact_channel


#### Load your own LFP data: ####

Resulting variables needed for subsequent analysis:
- LFP_array (np.ndarray, 6d): the raw LFP recording, containing all channels
- lfp_sig (np.ndarray, 1d): the channel containing the LFP signal from the hemisphere where the stimulation was delivered to generate artefacts
- LFP_rec_ch_names (list): names of all the channels, in a list (will be used to annotate cropped recording)
- sf_LFP (int): sampling frequency of intracerebral signal


In [ ]:
# choose LFP file
sub019 = main_class.PerceiveData(
    sub = "019", 
    incl_modalities=['streaming'],
    incl_session = ["fu24m"],
    incl_condition =['m0s0','m0s1','m1s0','m1s1'],
    incl_task = ["rest","fingerTap"],
    # incl_contact = ["RingL", "SegmInterR", "SegmIntraR"],
    import_json=False,
    warn_for_metaNaNs=False,
    allow_NaNs_in_metadata=False
)
LFP_rec = sub019.streaming.fu24m.m0s0.rest.run1.data

(LFP_array, 
 lfp_sig, 
 LFP_rec_ch_names, 
 sf_LFP) = _set_lfp_data(LFP_rec)

WITHOUT pyPerceive:

In [3]:
# Enter parameters:

sub_ID="Sub019 24MFU M0S0 rest"
fname_lfp="sub-20210415PStn_ses-2023040408103277_run-BrainSense20230404081800.mat"
ch_idx_lfp=0
fname_external="sub019_24mfu_M0S0_BrStr_Rest-20230404T101235.DATA.Poly5"
kernel = '2'
AUTOMATIC=False

#  Set saving path
project_path=os.getcwd()
print(project_path)

os.chdir('..')
main_directory=os.getcwd()
print(main_directory)

result_path = join(main_directory, "results")
print(result_path)

saving_path = join(result_path, sub_ID)
print(saving_path)
if not os.path.isdir(saving_path):
    os.makedirs(saving_path)



c:\Users\Juliette\Research\Projects\ReSync\scripts
c:\Users\Juliette\Research\Projects\ReSync
c:\Users\Juliette\Research\Projects\ReSync\results
c:\Users\Juliette\Research\Projects\ReSync\results\Sub019 24MFU M0S0 rest


In [12]:
os.chdir(main_directory)
dataset_lfp= _load_mat_file(sub_ID, fname_lfp, saving_path)
LFP_array, lfp_sig, LFP_rec_ch_names, sf_LFP = _load_data_lfp(sub_ID, dataset_lfp, ch_idx_lfp, saving_path)

Creating RawArray with float64 data, n_channels=6, n_times=45250
    Range : 0 ... 45249 =      0.000 ...   180.996 secs
Ready.


c:\Users\Juliette\anaconda3\envs\resync\Lib\site-packages\pymatreader\utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
c:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:37: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = read_raw_fieldtrip(
c:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:37: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  data = read_raw_fieldtrip(
c:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:37: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_02. Making it a MISC channel.
  data = read_raw_fieldtrip(
c:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.

#### Load your own external data: ####
(our external data recorder is a TMSi Data recorder.)

PM: NOTICE THE POP UP WINDOW AFTER RUNNING, TO SELECT THE FILE LOCATION

Resulting variables:
- external_file (np.ndarray, multi-dimensional): the complete external recording containing all channels recorded
- BIP_channel (np.ndarray, 1d): the channel containing the signal from the bipolar electrode used to pick up the artefacts on the IPG/cable
- external_rec_ch_names (list, same length as the number of channels in external_file): list of the channels names, to rename them accordingly after alignment
- sf_external (int): sampling frequency of the external data recorder

In [88]:
##  External
source_path = join(main_directory,'sourcedata')
TMSi_data = Poly5Reader(join(source_path, fname_external)) 
BIP_channel, external_file, external_rec_ch_names, sf_external, ch_index_external = _load_TMSi_artefact_channel(sub_ID, TMSi_data, fname_external, AUTOMATIC, saving_path)


Reading file  c:\Users\Juliette\Research\Projects\ReSync\sourcedata\sub019_24mfu_M0S0_BrStr_Rest-20230404T101235.DATA.Poly5
	 Number of samples:  846483 
	 Number of channels:  18 
	 Sample rate: 4000 Hz
Done reading data.
Creating RawArray with float64 data, n_channels=18, n_times=846483
    Range : 0 ... 846482 =      0.000 ...   211.620 secs
Ready.


c:\Users\Juliette\Research\Projects\ReSync\scripts\tmsi_poly5reader.py:94: RuntimeWarning: Channel names are not unique, found duplicates for: {'Y', 'X', 'Z'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=labels, sfreq=fs, ch_types=types_clean)


GENERATE EXAMPLE TRACES:

In [89]:
import numpy as np
from scipy.io import savemat

def replace_with_shuffled_values(signal, replace_start, replace_end, shuffle_start, shuffle_end):
    # Extract values from the specified part of the middle
    selected_values = signal[shuffle_start:shuffle_end]

    # Shuffle the selected values
    shuffled_values = np.random.permutation(selected_values)

    # Calculate the number of repetitions needed
    repetitions = (replace_end - replace_start) // len(shuffled_values)

    # Pad the shuffled values if needed
    shuffled_values_padded = np.tile(shuffled_values, repetitions + 1)[:replace_end - replace_start]

    # Replace the specified part of the signal with shuffled values
    randomized_signal = np.copy(signal)
    randomized_signal[replace_start:replace_end] = shuffled_values_padded

    return randomized_signal


In [92]:
signal1 =  np.copy(BIP_channel)

# Specify the start and end indices for the middle part to be replaced
middle_replace_start = 240000
middle_replace_end = 780000

# Specify the start and end indices for the part to be used for shuffling
shuffle_start = 510000
shuffle_end = 780000

# Replace the specified part of the signal with shuffled values
randomized_signal1 = replace_with_shuffled_values(signal1, middle_replace_start, middle_replace_end, shuffle_start, shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
plt.plot(BIP_channel, label='Original Signal')
plt.plot(randomized_signal1, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()

# now to shuffle beginning of the signal:
signal1 =  np.copy(randomized_signal1)

# Specify the start and end indices for the part to be replaced
beginning_replace_start = 0
beginning_replace_end = 174000

# Specify the start and end indices for the part to be used for shuffling
beginning_shuffle_start = 0
beginning_shuffle_end = 80000

# Replace the specified part of the signal with shuffled values
randomized_signal2 = replace_with_shuffled_values(signal1, beginning_replace_start, beginning_replace_end, beginning_shuffle_start, beginning_shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
plt.plot(BIP_channel, label='Original Signal')
plt.plot(randomized_signal2, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()

# Specify the file path for the .mat file
mat_file_path = 'C:\\Users\\Juliette\\Research\\Projects\\ReSync\\sourcedata\\randomized_external_signal.csv'

# Save the randomized signal as a .mat file
savemat(mat_file_path, {'randomized_external_signal': randomized_signal2})

## **READ ME** ##

Before starting the run_resync function, be careful to check that the config file is properly set. In particular, pay attention to:
- write the proper subject ID (to not overwrite previous analysis)
- by default use kernel "2", and set "real_index_LFP" to 0 for the first run. This can be adjusted if necessary before re-running.
- by default, set "consider_first_seconds_LFP", "consider_first_seconds_external" and "ignore_first_seconds_external" to null. 

Default parameters to adjust for our systems are:

- For TMSi SAGA with sf = 4000Hz or 4096Hz : thresh_external = -0.001, ch_name_BIP = "BIP 01"
- For TMSi SAGA with sf = 512Hz : thresh_external = -0.0005, ch_name_BIP = "BIP 01"
- For TMSi Porti with sf = 2048Hz : thresh_external = -2000, ch_name_BIP = "Bip25"

## PART 2: Align recordings: ##

In [ ]:
%matplotlib inline

(LFP_df_offset, 
 external_df_offset) = resync.run_resync(
    LFP_array=LFP_array,
    lfp_sig=lfp_sig,
    LFP_rec_ch_names=LFP_rec_ch_names,
    sf_LFP = sf_LFP,
    external_file=external_file,
    BIP_channel=BIP_channel,
    external_rec_ch_names=external_rec_ch_names,
    sf_external = sf_external,
    SHOW_FIGURES = True
)

In [ ]:
%matplotlib inline
#  Process/align recordings
LFP_df_offset, external_df_offset = run_resync(sub_ID, kernel, LFP_array, lfp_sig, LFP_rec_ch_names, sf_LFP, external_file, BIP_channel, external_rec_ch_names, sf_external, saving_path, SHOW_FIGURES = False)


#### Manual selection if no kernel is accurate enough: ####

WARNING: the plot is displayed in an interactive window, look for the pop up. Be careful that when you select the sample the user input window is not already opened and waiting for input, because it doesn't take into account the point selected after the opening of the user input window. If you're not sure, answer 'n' to the user input window and reselect the sample, then answer 'y' when it pops-up again.

WARNING 2: the sample chosen as 'start of the artefact' will automatically be the last point you clicked on. Be careful not to click anywhere else on the plot after having selected the proper sample.

In [ ]:
%matplotlib qt

closest_value_lfp = interact.select_sample(
    lfp_sig,
    sf_LFP
)

#### Run again the run_resync function once the real_art_time is adjusted: ####

In [ ]:
%matplotlib inline

(LFP_df_offset, 
 external_df_offset) = resync.run_resync(
    LFP_array=LFP_array,
    lfp_sig=lfp_sig,
    LFP_rec_ch_names=LFP_rec_ch_names,
    sf_LFP=sf_LFP,
    external_file=external_file,
    BIP_channel=BIP_channel,
    external_rec_ch_names=external_rec_ch_names,
    sf_external=sf_external,
    real_art_time_LFP=closest_value_lfp,
    SHOW_FIGURES=True
)

## PART 3 : Look for timeshift ##

Nothing needs to be changed in the following cells, they just need to be executed one after the other and follow instructions for interactive cells, until the timeshift is plotted:

In [ ]:
%matplotlib qt

#import settings
json_path = os.path.join(os.getcwd(), 'config')
json_filename = 'config.json'  # dont forget json extension
with open(os.path.join(json_path, json_filename), 'r') as f:
    loaded_dict =  json.load(f)

#set saving path
if not loaded_dict['saving_path']:
    saving_path = utils._define_folders()
else:
    saving_path = os.path.join(os.path.normpath(loaded_dict['saving_path']), loaded_dict['subject_ID'])
    if not os.path.isdir(saving_path):
        os.makedirs(saving_path)

# Reselect artefact channels in the aligned (= cropped) files:
LFP_channel_offset = LFP_df_offset.iloc[:, loaded_dict['LFP_ch_index']].to_numpy()  
BIP_channel_offset = external_df_offset.iloc[:, loaded_dict['BIP_ch_index']].to_numpy() 

# Generate new timescales:
LFP_timescale_offset_s = np.arange(
    start=0, 
    stop=len(LFP_channel_offset)/sf_LFP, 
    step=1/sf_LFP
)
external_timescale_offset_s = np.arange(
    start=0, 
    stop=len(external_df_offset)/sf_external, 
    step=1/sf_external
)

# detrend external recording with high-pass filter before processing:
filtered_external_offset = utils._filtering(BIP_channel_offset)


In [ ]:
last_artefact_lfp_x = interact.select_sample(LFP_channel_offset, sf_LFP)   # manually select last artefact in intracranial recording

In [ ]:
last_artefact_external_x = interact.select_sample(filtered_external_offset, sf_external)    # manually select last artefact in external recording

In [ ]:
timeshift_ms = (last_artefact_external_x - last_artefact_lfp_x)*1000

print(f'The timeshift at the last artefact is of {timeshift_ms:.2f} ms after a recording duration of {last_artefact_external_x:.2f} s.')

if abs(timeshift_ms) > 100:
    print('WARNING: the timeshift is unusually high, consider checking for packet loss in LFP data.')

In [ ]:
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle(str(loaded_dict['subject_ID']))
fig.set_figheight(6)
fig.set_figwidth(12)
ax1.axes.xaxis.set_ticklabels([])
ax2.set_xlabel('Time (s)')
ax1.set_ylabel('Intracerebral LFP channel (µV)')
ax2.set_ylabel('External bipolar channel (mV)')
ax1.set_xlim(last_artefact_external_x - 0.050, last_artefact_external_x + 0.1) 
ax2.set_xlim(last_artefact_external_x - 0.050, last_artefact_external_x + 0.1)
ax1.plot(
    LFP_timescale_offset_s,
    LFP_channel_offset,
    color='peachpuff',
    zorder=1
)
ax1.scatter(
    LFP_timescale_offset_s,
    LFP_channel_offset,
    color='darkorange',
    s=4,
    zorder=2
) 
ax1.axvline(
    x=last_artefact_lfp_x, 
    ymin=min(LFP_channel_offset), 
    ymax=max(LFP_channel_offset),
    color='black', 
    linestyle='dashed',
    alpha=.3
)
ax2.plot(
    external_timescale_offset_s,
    filtered_external_offset,
    color='paleturquoise',
    zorder=1
) 
ax2.scatter(
    external_timescale_offset_s,
    filtered_external_offset,
    color='darkcyan',
    s=4,
    zorder=2
) 
ax2.axvline(
    x=last_artefact_external_x,
    color='black',
    linestyle='dashed',
    alpha=.3
)
ax1.text(
    0.05, 
    0.85, 
    s='delay intra/exter: ' 
    + str(round(timeshift_ms, 2)) 
    + 'ms', 
    fontsize=14, 
    transform=ax1.transAxes
)
fig.savefig(
    saving_path 
    + '\\Fig8-Timeshift_Intracerebral and external recordings aligned_last artefact.png',
    bbox_inches='tight',
    dpi=1200
)


## PART 4 : Look for packet loss ##

WARNING: this only works for pyPerceive users for now. If you are not using pyPerceive to import your LFP data, consider changing the importation mode in resync.check_packet_loss to adapt it to your needs.

In [ ]:
json_fname = 'Report_Json_Session_Report_20230523T113952_ANOM.json'  # write json filename with extension
sub = '047' # write subject ID
resync.check_packet_loss(json_fname, sub)

## PLAYGROUND ##

##### ECG artefact verification #####

In [ ]:
%matplotlib inline

resync.ecg(
    LFP_df_offset,
    sf_LFP,
    external_df_offset,
    sf_external,
    xmin=0,
    xmax=3,
    SHOW_FIGURES = True
)